In [1]:
# create table public_for_2324.tt_tag_proportion as
#  select 'python' as lang, to_char(xx.creationdate, 'yyyy-mm-dd') as cdate , xx.id, xx.tag, xx.cnt, yy.tot_cnt, xx.cnt::FLOAT /yy.tot_cnt as pct
#    from (
#         select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag, 1 as cnt
#          from public_for_2324.posts
#          where posttypeid = '1'
#            and creationdate between '2021-11-30' and '2024-12-02'
#            and tags like '%<python>%'
#             ) xx,
#         (
#             select x.id, count(x.tag) as tot_cnt
#                from (select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag
#                        from public_for_2324.posts
#                      where posttypeid = '1'
#                        and creationdate between '2021-11-30' and '2024-12-02'
#                        and tags like '%<python>%') x
#              group by x.id

#             )yy
#  where xx.id = yy.id
#    and xx.tag = 'python'
#  ;


# insert into public_for_2324.tt_tag_proportion
#  select 'c++' as lang, to_char(xx.creationdate, 'yyyy-mm-dd') as cdate , xx.id, xx.tag, xx.cnt, yy.tot_cnt, xx.cnt::FLOAT /yy.tot_cnt as pct
#    from (
#         select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag, 1 as cnt
#          from public_for_2324.posts
#          where posttypeid = '1'
#            and creationdate between '2021-11-30' and '2024-12-02'
#            and tags like '%<c++>%'
#             ) xx,
#         (
#             select x.id, count(x.tag) as tot_cnt
#                from (select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag
#                        from public_for_2324.posts
#                      where posttypeid = '1'
#                        and creationdate between '2021-11-30' and '2024-12-02'
#                        and tags like '%<c++>%') x
#              group by x.id

#             )yy
#  where xx.id = yy.id

#  ;

# insert into public_for_2324.tt_tag_proportion
#   select 'java' as lang, to_char(xx.creationdate, 'yyyy-mm-dd') as cdate , xx.id, xx.tag, xx.cnt, yy.tot_cnt, xx.cnt::FLOAT /yy.tot_cnt as pct
#     from (
#          select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag, 1 as cnt
#           from public_for_2324.posts
#           where posttypeid = '1'
#             and creationdate between '2021-11-30' and '2024-12-02'
#             and tags like '%<java>%'
#              ) xx,
#          (
#              select x.id, count(x.tag) as tot_cnt
#                 from (select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag
#                         from public_for_2324.posts
#                       where posttypeid = '1'
#                         and creationdate between '2021-11-30' and '2024-12-02'
#                         and tags like '%<java>%') x
#               group by x.id
#              )yy
#   where xx.id = yy.id;

#  insert into public_for_2324.tt_tag_proportion
#   select 'vba' as lang, to_char(xx.creationdate, 'yyyy-mm-dd') as cdate , xx.id, xx.tag, xx.cnt, yy.tot_cnt, xx.cnt::FLOAT /yy.tot_cnt as pct
#     from (
#          select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag, 1 as cnt
#           from public_for_2324.posts
#           where posttypeid = '1'
#             and creationdate between '2021-11-30' and '2024-12-02'
#             and tags like '%<vba>%'
#              ) xx,
#          (
#              select x.id, count(x.tag) as tot_cnt
#                 from (select creationdate, id, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag
#                         from public_for_2324.posts
#                       where posttypeid = '1'
#                         and creationdate between '2021-11-30' and '2024-12-02'
#                         and tags like '%<vba>%') x
#               group by x.id
#              )yy
#   where xx.id = yy.id
#   ;

In [2]:

#  create table public_for_2324.tt_q_tag_cnt_01 as
#   select x.id
#   	, to_char(x.creationdate, 'yyyy-mm-dd') as c_date
#   	, (regexp_matches(x.tags, '<(ruby|scala|rust|go|bash|shell|objective-c|python|r|typescript|swift|c#|sql|assembly|kotlin|c\+\+|javascript|vba|c|html|css|java|perl|haskell|julia|php|dart|clojure|f#|elixir|erlang|lisp|groovy|crystal-lang|powershell|cobol|delphi|matlab)>', 'g'))[1] AS matched_tags
#   	, x.tag
#     from (
#   	select id, creationdate, tags, replace(replace(unnest(string_to_array(tags, '><')), '<', ''), '>', '') tag
#     from public_for_2324.posts
#   where posttypeid ='1'
#     and (tags like '%<ruby>%' or tags like '%<scala>%' or tags like '%<rust>%' or tags like '%<go>%' or tags like '%<bash>%' or tags like '%<shell>%' or tags like '%<objective-c>%' or tags like '%<python>%' or tags like '%<r>%' or tags like '%<typescript>%' or tags like '%<swift>%' or tags like '%<c#>%' or tags like '%<sql>%' or tags like '%<assembly>%' or tags like '%<kotlin>%' or tags like '%<c++>%' or tags like '%<javascript>%' or tags like '%<vba>%' or tags like '%<c>%' or tags like '%<html>%' or tags like '%<css>%' or tags like '%<java>%' or tags like '%<perl>%' or tags like '%<haskell>%' or tags like '%<julia>%' or tags like '%<php>%' or tags like '%<dart>%' or tags like '%<clojure>%' or tags like '%<f#>%' or tags like '%<elixir>%' or tags like '%<erlang>%' or tags like '%<lisp>%' or tags like '%<groovy>%' or tags like '%<crystal-lang>%' or tags like '%<powershell>%' or tags like '%<cobol>%' or tags like '%<delphi>%' or tags like '%<matlab>%')
#     and creationdate between '2021-01-01' and '2024-12-03'
#     ) x
#   ;

#  create table public_for_2324.tt_q_tag_cnt_02 as
#    select a.id, a.c_date, a.matched_tags, a.tag, b.tot_cnt, CAST(1 as numeric)/b.tot_cnt as prop_for_t
#      from public_for_2324.tt_q_tag_cnt_01 a
#         , (select id, matched_tags, count(*) as tot_cnt
#  		   from public_for_2324.tt_q_tag_cnt_01
#  		  group by id, matched_tags
#         ) b
#    where a.id = b.id
#      and a.matched_tags = b.matched_tags;


In [3]:
import sys, os
p = os.path.abspath('..')
pp = os.path.abspath('../..')
ppp = os.path.abspath('../visualization')
sys.path.insert(1, p)
sys.path.insert(1, pp)
sys.path.insert(1, ppp)

In [4]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf
import string
import re
import config.config as conf
from constants import CONSTANTS
import json

In [5]:
# lang='python'
lang='vba'

In [6]:
conn = psycopg2.connect(dbname  =   conf.database_user['dbname'], 
                        user    =   conf.database_user['user'], 
                        password=   conf.database_user['password'], 
                        host    =   conf.database_user['host'], 
                        port    =   conf.database_user['port'])

In [7]:
lang_tag_dict = {'python' : 'python',
                'cpp': 'c++',
                'java':'java',
                'vba':'vba'
                }

In [8]:
for idx in range(len(CONSTANTS.monthly_timestamps)-1):
    st_dt = CONSTANTS.monthly_timestamps[idx].replace('.','-')
    end_dt = CONSTANTS.monthly_timestamps[idx+1].replace('.','-')

    cur = conn.cursor()
    sql = f"""select c_date, matched_tags, tag, sum(prop_for_t) as cnt from public_for_2324.tt_q_tag_cnt_02 where c_date >=  '{str(st_dt)}'  and c_date < '{str(end_dt)}'  and matched_tags =  '{lang_tag_dict[lang]}' group by c_date, matched_tags, tag """

    cur.execute(sql)
    inserted_data = cur.fetchall()
    type(inserted_data)
    cur.close()
    

    dict_q = [{'creationdate' : x[0], 
            'matched_tag' : x[1],
            'tag' : x[2],
            'cnt' : float(x[3]) 
            } for x in inserted_data]
    # json_str = json.dumps(dict_q, default=str, ensure_ascii=False, indent=2)

    with open(f"../../visualization/result/tag/run_id_2/data/{lang}/{idx}.json", "w", encoding="utf-8") as f:
        json.dump(dict_q, f, ensure_ascii=False, indent=2)



In [9]:
cur = conn.cursor()
cur.execute(f"""select tag, count(*) as count from public_for_2324.tt_q_tag_cnt_02 where matched_tags = '{lang_tag_dict[lang]}' group by tag """)
inserted_data = cur.fetchall()
cur.close()
    

dict_q = { x[0]: {"count" : x[1]} for x in inserted_data}
print(type(dict_q))

with open(f"../../visualization/result/tag/run_id_2/data/tag_info_{lang}.json", "w", encoding="utf-8") as f:
    json.dump(dict_q, f, ensure_ascii=False, indent=2)



<class 'dict'>


In [10]:
conn.close()